## WKT to Segmentation Mask

This notebook illustrates an example to convert segmentation WKTs into grayscale image mask.

This example uses:  
- `PIL` library for image I/O and drawing mask
- `shapely` for pre-processing WKTs and generating polygons

(If you want to use these WKTs from us, don't forget to convert them (see Recipe @TODO))

In [ ]:
# install dependencies
! pip install -r requirements.txt

In [ ]:
# imports
import json
import numpy as np

from PIL import Image, ImageDraw

import shapely.wkt
import shapely.geometry

In [ ]:
# inputs paths
JSON_WKT_PATH = './example/input/example_mask_wkt.json'
BASE_IMAGE_FILE_PATH = './example/input/example_base.png'
# output path
MASK_IMAGE_FILE_PATH = './example/output/example_mask.png'

### segmentation wkts to grayscale mask

In [ ]:
# load wkts from json
with open(JSON_WKT_PATH) as f:
    list_of_wkt_masks = json.load(f)
list_of_wkt_masks

In [ ]:
# load base image
base_image = Image.open(BASE_IMAGE_FILE_PATH)
# prepare blank canvas
canvas = np.zeros((int(base_image.height), int(base_image.width)))

# draw mask
mask_image = Image.fromarray(np.uint8(canvas))
for wkt_mask in list_of_wkt_masks:
    mask_polygon = shapely.wkt.loads(wkt_mask["wkt"])
    if mask_polygon.is_empty:
            continue
    mask_poly_coords = list(mask_polygon.exterior.coords)
    ImageDraw.Draw(mask_image).polygon(mask_poly_coords, fill = wkt_mask['intensity'])
    
# save mask
mask_image.save(MASK_IMAGE_FILE_PATH)

### visualize generated mask on base image

In [ ]:
from PIL import ImageFont
from matplotlib import font_manager

font = font_manager.FontProperties(family='sans-serif', weight='bold')
font = ImageFont.truetype(font_manager.findfont(font), 48)

# create new layer
layer = Image.new('RGBA', base_image.size, (0, 0, 0, 0))
layer.paste(mask_image.convert('RGBA'), (0,0))
# create a copy of the layer
layer2 = layer.copy()
# put alpha on the copy
layer2.putalpha(128)
# merge layers with mask
layer.paste(layer2, layer)
# overlay
composite_image = Image.alpha_composite(base_image, layer)

# show overlay
ImageDraw.Draw(base_image).text((100, 100), "Base Image", (255,255,255), font=font)
ImageDraw.Draw(mask_image := mask_image.convert("RGBA")).text((100, 100), "Mask Image", (255,255,255), font=font)
ImageDraw.Draw(composite_image).text((100, 100), "Mask Overlay", (255,255,255), font=font)
overlay_strip = Image.fromarray(np.hstack((np.array(base_image), 
                                           np.array(mask_image), 
                                           np.array(composite_image))))
overlay_strip